In [48]:
import pandas as pd
import numpy as np
import seaborn
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException

In [53]:
def scrapeComments(route_id):
    '''
    Accepts a route id parsed from URL, if comments exists, returns comments in list 
    '''
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='comments-Climb-Route-" + route_id + "']/div[2]/div[2]")))
    route_com = driver.find_element_by_xpath("//*[@id='comments-Climb-Route-" + route_id + "']/div[2]/div[2]")
    has_comments = None 
    try:
        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "comment-body")))
        has_comments = True 
    except TimeoutException:  
            has_comments = False 
            print(route_id, 'has no comments')
    if has_comments == True:
        comment_body = route_com.find_elements_by_class_name('comment-body')
        func_comments = []
        for i in comment_body: 
            func_comments.append(i.text)
        return(func_comments)


In [3]:
#Read in Config txt file as dict
with open(r"C:\Users\drpow\Documents\Personal Projects\positively_climbable\config.txt", "r") as config_txt:
    config_txt = eval(config_txt.read())
config = config_txt

#location 
state = str(config_txt['state_location'])
sub_location = str(config_txt['sub_location'])

#mapping rock route type to HTML id + map other settings to vars 
route_type = config["route_type"].lower()
diff_rating_range = config["diff_rating_range"]

rock_type_dict = {"sport":"check_is_sport_climb", "trad":"check_is_trad_climb", "toprope" : "check_is_top_rope"}
rock_type = config["rock_route_type_exclude"]

In [61]:
driver = webdriver.Chrome(r"C:\Users\drpow\Documents\Personal Projects\positively_climbable\chromedriver.exe")
driver.get("http://www.mountainproject.com")
wait = WebDriverWait(driver, 3)

# Apply Search Settings 

In [62]:
#apply config settings to "route finder"
select_type = Select(driver.find_element_by_id("type")) 
select_type.select_by_value(route_type)

#lb = lower bound, ub = upper bound
if route_type =='rock': 
    Select(driver.find_element_by_id("diffMinrock")).select_by_visible_text(diff_rating_range[0])
    Select(driver.find_element_by_id("diffMaxrock")).select_by_visible_text(diff_rating_range[1])
    for rock_toggle in rock_type: 
        if rock_type[rock_toggle] == True:
            driver.find_element_by_id(rock_type_dict[rock_toggle]).click()
            
if state != '': 
    driver.find_element_by_xpath('//*[@id="routeFinderForm"]/table/tbody/tr[4]/td[2]/a').click()
    wait.until(EC.element_to_be_clickable((By.LINK_TEXT, state))).click()
    if sub_location != '': 
        wait.until(EC.element_to_be_clickable((By.LINK_TEXT, sub_location))).click()
    driver.find_element_by_id("select-area").click()
        
        
driver.find_element_by_xpath("//*[@id='routeFinderForm']/table/tbody/tr[5]/td/input").click()
wait.until(EC.element_to_be_clickable((By.LINK_TEXT, "View All"))).click()

In [54]:
driver.find_element_by_xpath("//*[@id='route-page']/div/div[1]/h1").text

'John Fischer Memorial Route'

In [63]:
num_routes = (driver.find_element_by_xpath("//*[@id='body-climb']/div[6]/div/div[2]/div/div[1]").text).split('Results 1 to ')[1].split(' ')[0]
comm_dict = {}
for route in range(2, int(num_routes)+2): 
    driver.find_element_by_xpath("//*[@id='body-climb']/div[6]/div/div[3]/div/div/table[2]/tbody/tr["+str(route)+"]/td[1]/a").click() 
    url_split = driver.current_url.split('/')
    route_id = url_split[4]
    route_name = url_split[5]
    comm_dict[route_name] = scrapeComments(route_id)
    driver.back()


106783221 has no comments


KeyboardInterrupt: 

In [66]:
len(comm_dict)

64

In [ ]:
//*[@id="comments-Climb-Route-105912080"]

In [48]:
route_table = driver.find_element_by_xpath("//*[@id='body-climb']/div[6]/div/div[3]/div/div/table[1]")
table_rows = route_table.find_elements_by_xpath("//tbody")


table_rows[0].click()

In [10]:
#comments scrape 

test = driver.find_element_by_xpath("//*[@id='comments-Climb-Route-" + route_id + "']/div[2]/div[2]")
test2 = test.find_elements_by_class_name('comment-body')
for i in test2: 
    print(i.text, end = '\t NEWB')

NameError: name 'route_id' is not defined

In [71]:
"//*[@id='comments-Climb-Route-" + str(105843581) + "']/div[2]/div[2]"

"//*[@id='comments-Climb-Route-105843581']/div[2]/div[2]"

In [55]:
//*[@id="comments-Climb-Route-105912080"]/div[2]/div[2]

In [ ]:
//*[@id="comments-Climb-Route-105843581"]/div[2]/div[2]